In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from pprint import pprint

import src
from src.reload import deep_reload

In [12]:
icevs = {
    'pacifica': {
        'gallons_gas': 19,
        'mpg': 25,
    },
    'prius': {
        'gallons_gas': 11.3,
        'mpg': 56,
    },
    'golf': {
        'gallons_gas': 13.2,
        'mpg': 41,
    },
}

In [13]:
for key, value in icevs.items():

    value['energy_consumption'] = 1 / value['mpg'] / 1.6094 * 33.7 * 3.6e3
    value['ess_capacity'] = value['gallons_gas'] * 33.7
    value['range'] = value['ess_capacity'] * 3.6e3 / value['energy_consumption']

In [14]:
icevs

{'pacifica': {'gallons_gas': 19,
  'mpg': 25,
  'energy_consumption': 3015.2851994532125,
  'ess_capacity': 640.3000000000001,
  'range': 764.4650000000001},
 'prius': {'gallons_gas': 11.3,
  'mpg': 56,
  'energy_consumption': 1346.1094640416127,
  'ess_capacity': 380.81000000000006,
  'range': 1018.4283200000002},
 'golf': {'gallons_gas': 13.2,
  'mpg': 41,
  'energy_consumption': 1838.588536251959,
  'ess_capacity': 444.84000000000003,
  'range': 871.0072799999998}}

In [30]:
gas_supply_rate = 7 / 60 * 33.7 * 3.6e6

{k: gas_supply_rate / v['energy_consumption'] * 60 / 1e3 for k, v in icevs.items()}

{'pacifica': 281.64500000000004, 'prius': 630.8848, 'golf': 461.89779999999996}

In [44]:
gas_energy_price = 5.138 / 33.7 / 3.6e3

{k: v['energy_consumption'] * gas_energy_price * 100 for k, v in icevs.items()}

{'pacifica': 12.769976388716293,
 'prius': 5.700882316391201,
 'golf': 7.786570968729448}

In [61]:
gas_energy_price = 3.609 / 33.7 / 3.6e3

{k: v['energy_consumption'] * gas_energy_price * 100 for k, v in icevs.items()}

{'pacifica': 8.969802410836337,
 'prius': 4.0043760762662215,
 'golf': 5.469391713924597}

In [96]:
graph = src.graph.graph_from_json('Outputs/graph_full.json')

cities = [k for k, v in graph._node.items() if v['type'] == 'city']

In [140]:
cities_sg = src.graph.subgraph(graph, cities)

In [141]:
def icev_trip_time(edge, vehicle, supply_rate, base_delay = 60, range_usage_ratio = .9):

    effective_capacity = vehicle['ess_capacity'] * range_usage_ratio * 3.6e6
    effective_range = vehicle['range'] * range_usage_ratio * 1e3
    
    number_of_stops = edge['distance'] // effective_range
    remainder_distance = edge['distance'] % effective_range
    edge['supply_time'] = (
        number_of_stops * effective_capacity / supply_rate +
        number_of_stops * base_delay
    )

    # print(edge['supply_time'])

    edge['total_time'] = edge['time'] + edge['supply_time']
    edge['number_of_stops'] = number_of_stops

    return edge

In [142]:
supply_rate = 7 / 60 * 33.7 * 3.6e6
base_delay = 600

values = {}

for key, vehicle in icevs.items():
    
    values[key] = {}
    
    for source, adj in cities_sg._adj.items():

        values[key][source] = {}
        
        for target, edge in adj.items():
            
            values[key][source][target] = icev_trip_time(
                edge.copy(), vehicle, supply_rate, base_delay = base_delay
            )

In [163]:
deep_reload(src)

graph = src.graph.graph_from_json('Outputs/graph_empty.json')

cities = src.graph.graph_from_json('cities.json')

pop_adj = sum([v['population'] for k, v in cities._node.items()]) / len(cities)

weighted = {k: v['population'] for k, v in cities._node.items()}

relative = {k: v['population'] / pop_adj for k, v in cities._node.items()}

unweighted = {k: 1 for k, v in cities._node.items()}

In [168]:
src.routing.impedance(
    values['prius'],
    field = 'total_time',
    origins = relative,
    destinations = relative,
    constant = 1 / 3600
)

4.130946393010607

In [169]:
src.routing.impedance(
    values['golf'],
    field = 'total_time',
    origins = relative,
    destinations = relative,
    constant = 1 / 3600
)

4.151156828756136

In [167]:
src.routing.impedance(
    values['pacifica'],
    field = 'total_time',
    origins = relative,
    destinations = relative,
    constant = 1 / 3600
)

4.161197186840438

In [190]:
src.routing.impedance(
    values['prius'],
    field = 'time',
    origins = relative,
    destinations = relative,
    constant = 1 / 3600
)

4.118334452421349

In [191]:
src.routing.impedance(
    values['golf'],
    field = 'time',
    origins = relative,
    destinations = relative,
    constant = 1 / 3600
)

4.118334452421349

In [192]:
src.routing.impedance(
    values['pacifica'],
    field = 'time',
    origins = relative,
    destinations = relative,
    constant = 1 / 3600
)

4.118334452421349

In [194]:
src.routing.impedance(
    values['pacifica'],
    field = 'total_time',
    origins = {'Sacramento': 1},
    destinations = relative,
    constant = 1 / 3600
)

5.186834177368633

In [195]:
src.routing.impedance(
    values['pacifica'],
    field = 'total_time',
    origins = {'San Diego': 1},
    destinations = relative,
    constant = 1 / 3600
)

4.135867919622814

In [157]:
print(src.routing.road_trip_accessibility(
    values['pacifica'],
    field = 'time',
) / 3600)

print(src.routing.road_trip_accessibility(
    values['golf'],
    field = 'time',
) / 3600)

print(src.routing.road_trip_accessibility(
    values['prius'],
    field = 'time',
) / 3600)

5.439653657330323
5.439653657330323
5.439653657330323


In [150]:
max_trip_length = 0
max_length_trip = ''

for source, adj in cities_sg._adj.items():
        
    for target, edge in adj.items():

        if edge['distance'] > max_trip_length:

            max_trip_length = edge['distance']
            max_length_trip = (source, target)

max_length_trip, max_trip_length / 1e3

(('Crescent City', 'I-10 to Phoenix (State Line)'), 1532.66)

In [155]:
total_stops = {}

for veh, val in values.items():

    total_stops[veh] = 0

    for source, adj in val.items():
        for target, arc in adj.items():

            total_stops[veh] += arc['number_of_stops']

total_stops, {k: v / 14 ** 2 for k, v in total_stops.items()}

({'pacifica': 84.0, 'prius': 38.0, 'golf': 60.0},
 {'pacifica': 0.42857142857142855,
  'prius': 0.19387755102040816,
  'golf': 0.30612244897959184})